## Load and Clean Datasets and Create NBA Players Database

In this notebook, we will load all of the datasets obtained from the internet through APIs, Kaggle, and other methods and clean the datasets for usage. We will use pandas and SQL to load and merge the datasets to create one master dataset for prediction.

The database will feature NBA player stats, injury information, jersey sales information, team valuation information, and salaries.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import os

# Define the path for the SQLite database
sqlite_db_path = 'nba_players.db'

# Remove existing SQLite DB if you want a fresh start (optional)
if os.path.exists(sqlite_db_path):
    os.remove(sqlite_db_path)

# Create a connection string for SQLite
connection_string = f'sqlite:///{sqlite_db_path}'

# Create the SQLite engine
engine = create_engine(connection_string)

#### Load and Clean NBA Advanced Stats

This dataset includes all of the advanced stats for all NBA players from 2019-2024.

In [ ]:
# Load the data from the CSV file
df = pd.read_csv(r"C:/Users/mdani/OneDrive/Desktop/nba_players/NBA Advanced Stats(2019 - 2024).csv")
print(f"Loaded NBA Advanced Stats with {df.shape[0]} rows.")

# Initial exploration of the data
print("First 5 rows of the NBA Advanced Stats dataset:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nSummary Statistics:")
print(df.describe())

print("\nMissing values in each column:")
print(df.isnull().sum())

print("\nColumn names:")
print(df.columns)

## DATA CLEANING

# Drop rows with missing values in 'Player' or 'Season'
df_cleaned = df.dropna(subset=['Player', 'Season'])
print(f"After dropping missing values: {df_cleaned.shape[0]} rows.")

# Remove duplicates: Keep the row with the greater 'MP' value
df_cleaned = df_cleaned.sort_values(by='MP', ascending=False).drop_duplicates(subset=['Player', 'Season'], keep='first')
print(f"After removing duplicates: {df_cleaned.shape[0]} rows.")

# Create a unique player ID based on the combination of 'Season' and 'Player'
df_cleaned['Unnamed: 0'] = df_cleaned['Season'].astype(str) + "_" + df_cleaned['Player']

# Rename columns for consistency
df_cleaned = df_cleaned.rename(columns={'Unnamed: 0': 'Player_ID'})
df_cleaned = df_cleaned.rename(columns={'MP': 'Total MP'})

# Sort by 'Player_ID' in ascending order and reset index
df_cleaned = df_cleaned.sort_values(by='Player_ID').reset_index(drop=True)
print(df_cleaned.head())

# Import the DataFrame into the SQLite database
df_cleaned.to_sql('nba_advanced_stats', engine, if_exists='replace', index=False)
print("Imported 'nba_advanced_stats' into SQLite database.\n")

#### Load and Clean NBA Per Game Playoff Stats

This dataset has stats for all players from 2019-2024 who were part of playoff games for each year.

The primary key is Player_ID.

In [ ]:
# Load the data
df2 = pd.read_csv(r"C:/Users/mdani/OneDrive/Desktop/nba_players/NBA Per Game PLAYOFF Stats(2019 - 2024).csv")
print(f"Loaded NBA Per Game Playoff Stats with {df2.shape[0]} rows.")

# Initial exploration
print("First 5 rows of the NBA Per Game Playoff Stats dataset:")
print(df2.head())

print("\nDataset Info:")
print(df2.info())

print("\nSummary Statistics:")
print(df2.describe())

print("\nMissing values in each column:")
print(df2.isnull().sum())

print("\nColumn names:")
print(df2.columns)

## DATA CLEANING

# Drop rows with missing values in 'Player' or 'Season'
df2_cleaned = df2.dropna(subset=['Player', 'Season'])
print(f"After dropping missing values: {df2_cleaned.shape[0]} rows.")

# Remove duplicates: Keep the first occurrence
df2_cleaned = df2_cleaned.drop_duplicates(subset=['Player', 'Season'], keep='first')
print(f"After removing duplicates: {df2_cleaned.shape[0]} rows.")

# Create a unique player ID
df2_cleaned['Unnamed: 0'] = df2_cleaned['Season'].astype(str) + "_" + df2_cleaned['Player']

# Rename columns for consistency
df2_cleaned = df2_cleaned.rename(columns={'Unnamed: 0': 'Player_ID'})
df2_cleaned = df2_cleaned.rename(columns={'MP': 'MPG'})

# Sort and reset index
df2_cleaned = df2_cleaned.sort_values(by='Player_ID').reset_index(drop=True)
print(df2_cleaned.head())

# Import into SQLite
df2_cleaned.to_sql('nba_playoff_stats', engine, if_exists='replace', index=False)
print("Imported 'nba_playoff_stats' into SQLite database.\n")


#### Load and Clean NBA Per Game Stats

This dataset includes NBA statistics that are not in the advanced stats. The idea is to have the most comprehensive dataset to account for all performance factors dictating a contract. 

Once again, the primary key is Player_ID

In [ ]:
# Load the data
df3 = pd.read_csv(r"C:/Users/mdani/OneDrive/Desktop/nba_players/NBA Per Game Stats(2019 - 2024).csv")
print(f"Loaded NBA Per Game Stats with {df3.shape[0]} rows.")

# Initial exploration
print("First 5 rows of the NBA Per Game Stats dataset:")
print(df3.head())

print("\nDataset Info:")
print(df3.info())

print("\nSummary Statistics:")
print(df3.describe())

print("\nMissing values in each column:")
print(df3.isnull().sum())

print("\nColumn names:")
print(df3.columns)

## DATA CLEANING

# Drop rows with missing values in 'Player' or 'Season'
df3_cleaned = df3.dropna(subset=['Player', 'Season'])
print(f"After dropping missing values: {df3_cleaned.shape[0]} rows.")

# Remove duplicates: Keep the first occurrence
df3_cleaned = df3_cleaned.drop_duplicates(subset=['Player', 'Season'], keep='first')
print(f"After removing duplicates: {df3_cleaned.shape[0]} rows.")

# Create a unique player ID
df3_cleaned['Unnamed: 0'] = df3_cleaned['Season'].astype(str) + "_" + df3_cleaned['Player']

# Rename columns for consistency
df3_cleaned = df3_cleaned.rename(columns={'Unnamed: 0': 'Player_ID'})
df3_cleaned = df3_cleaned.rename(columns={'MP': 'MPG'})

# Sort and reset index
df3_cleaned = df3_cleaned.sort_values(by='Player_ID').reset_index(drop=True)
print(df3_cleaned.head())

# Import into SQLite
df3_cleaned.to_sql('nba_per_game_stats', engine, if_exists='replace', index=False)
print("Imported 'nba_per_game_stats' into SQLite database.\n")


#### Load and Clean NBA Player Injury Stats

This dataset gave information about which players got injured, when they got injured, and what their injury was. The goal of cleaning and transforming this dataset was to provide the total injury days per player from 2019-2024 to show their availabilities during the season. We believe greater injury days should reduce the salary price. 

In [ ]:
# Load the data
df4 = pd.read_csv(r"C:/Users/mdani/OneDrive/Desktop/nba_players/NBA Player Injury Stats(2019 - 2024).csv")
print(f"Loaded NBA Player Injury Stats with {df4.shape[0]} rows.")

print(df4.columns)

import pandas as pd
from datetime import datetime

df4['Date'] = pd.to_datetime(df4['Date'], errors='coerce')

df4['Player'] = df4['Acquired'].fillna(df4['Relinquished']) # Identify the player name in each row

# Store total injury days per player
player_injury_days = {}

for player in df4['Player'].unique():
    # Filter rows for this player and sort by Date
    player_data = df4[(df4['Relinquished'] == player) | (df4['Acquired'] == player)].sort_values(by='Date')

    total_injury_days = 0
    injury_start = None

    for _, row in player_data.iterrows():
        if row['Relinquished'] == player:
            # Start a new injury period if no ongoing injury
            if injury_start is None:
                injury_start = row['Date']
                print(f"Start injury: {injury_start} for {player}")
        elif row['Acquired'] == player and injury_start is not None:
            # End the current injury period
            injury_end = row['Date']
            print(f"End injury: {injury_end} for {player}")

            # Calculate injury days
            if injury_start < injury_end:
                injury_days = (injury_end - injury_start).days
                total_injury_days += injury_days
                print(f"Added {injury_days} days for {player}")
            else:
                print(f"Invalid period: {injury_start} -> {injury_end} for {player}")

            # Reset injury_start
            injury_start = None

    # Handle open-ended injuries
    if injury_start is not None:
        # End on the next July 6 or 2024-11-24 --> this either represents the end of the trade deadline or the latest day of data provided
        year = injury_start.year
        cutoff_date = datetime(year, 7, 6) if injury_start < datetime(year, 7, 6) else datetime(year + 1, 7, 6)
        injury_end = min(cutoff_date, datetime(2024, 11, 24))

        # Add only valid days
        if injury_start < injury_end:
            injury_days = (injury_end - injury_start).days
            total_injury_days += injury_days
            print(f"Open-ended injury: Added {injury_days} days for {player}")
        else:
            print(f"Invalid open-ended period: {injury_start} -> {injury_end} for {player}")

    # Store total injury days for this player
    player_injury_days[player] = total_injury_days
    print(f"Total days for {player}: {total_injury_days}\n")

print(player_injury_days)

## Clean the player names since there are unwanted words and characters in there

import re

# Clean the player names in the player_injury_days dictionary
cleaned_player_injury_days = {}

for player, injury_days in player_injury_days.items():
    cleaned_name = re.sub(r'\(.*?\)', '', player)
    # Keep only the name after the last '/'
    cleaned_name = cleaned_name.split('/')[-1]
    cleaned_player_injury_days[cleaned_name] = injury_days

print(cleaned_player_injury_days)


df4['Injury_Days'] = df4['Player'].map(cleaned_player_injury_days)

print(df4.head())

# Import into SQLite
df4.to_sql('nba_injury_days', engine, if_exists='replace', index=False)
print("Imported 'nba_injury_days' into SQLite database.\n")

#### Load and Clean NBA Salaries

We want to load this dataset to get all of the yearly salaries for each player in the dataset and add the salaries where the Player_ID matches. Therefore, there is appropriate information for analysis on a yearly basis for the entire NBA.

In [ ]:
# Load the data
df5 = pd.read_csv(r"C:/Users/mdani/OneDrive/Desktop/nba_players/NBA Salaries(2019-2024).csv")
print(f"Loaded NBA Salaries with {df5.shape[0]} rows.")

print(df5.head(20))

# Transform
df5_melted = pd.melt(
    df5,
    id_vars=["Unnamed: 0", "playerName"],
    var_name="Year",
    value_name="Salary"
)

# We want the latest salary as this includes bonuses
df5_melted["Year"] = df5_melted["Year"].str.replace(r"\(\*\)", "", regex=True).str.strip()
df5_melted["Year"] = df5_melted["Year"].str[:4]  

df5_melted.head()
df5_melted["Unnamed: 0"] = df5_melted["Year"] + "_" + df5_melted["playerName"]

df5_melted = df5_melted.rename(columns={'Unnamed: 0': 'Player_ID'})

# Remove "$" and "," from Salary column and convert it to numeric
df5_melted["Salary"] = df5_melted["Salary"].str.replace("[$,]", "", regex=True).astype(float)

# Sort by playerName, Year, and then keep the last value for duplicates
df5_melted = df5_melted.sort_values(by=["playerName", "Year"]).groupby(["playerName", "Year"]).nth(-1).reset_index()

# Reorder the columns
df5_melted = df5_melted[["Player_ID", "Year", "playerName", "Salary"]]
print("Cleaned NBA Salaries DataFrame:")
print(df5_melted.head(10))

# Import into SQLite
df5_melted.to_sql('nba_salaries', engine, if_exists='replace', index=False)
print("Imported 'nba_salaries' into SQLite database.\n")

#### Load and Clean NBA Team Valuations

This dataset gives information about the values of each team. Some teams or worth more than others which is a factor in the contract sizes they are able to offer. Therefore, we need to take them into account and add that for each player where the Team code matches in each dataset. 

In [ ]:
# Load the data
df6 = pd.read_csv(r"C:/Users/mdani/OneDrive/Desktop/nba_players/NBA_Team_Valuations_2024.csv")
print("Loaded NBA Team Valuations:")
print(df6)

# Calculate mean and standard deviation for Valuation
mean_valuation = df6["Valuation (in billions)"].mean()
std_valuation = df6["Valuation (in billions)"].std()

# Calculate z-scores
df6["Z-Score"] = (df6["Valuation (in billions)"] - mean_valuation) / std_valuation

# Apply scaling factor and set bounds for the multipliers as it should not affect the contract too much
# This helps for standardization
scaling_factor = 0.1  
df6["Valuation_Multiplier"] = 1 + (scaling_factor * df6["Z-Score"])
df6["Valuation_Multiplier"] = df6["Valuation_Multiplier"].clip(lower=0.8, upper=1.2)
print(df6.head())

# Import into SQLite
df6.to_sql('nba_team_valuations', engine, if_exists='replace', index=False)
print("Imported 'nba_team_valuations' into SQLite database.\n")


#### Load and Clean NBA Jersey Sales

This gives information of which players were in the Top 15 for Jersey Sales. This contributes to their contracts as a bigger brand attracts more attention to the team. This increases sales and increases the value of the team. Therefore, players who have a lot of jersey sales receive larger contracts. 

This only applies to players for 2023 as this is the latest branding. 

In [ ]:
# Load the data
df7 = pd.read_csv(r"C:/Users/mdani/OneDrive/Desktop/nba_players/Top_NBA_Jersey_Sales.csv")
print("Loaded Top NBA Jersey Sales:")
print(df7.head())

df_sub = df7[["2023-2024", "Multiplier"]].copy()
df_sub["Player_ID"] = "2023_" + df_sub["2023-2024"]

# Rename columns
df_sub = df_sub.rename(columns={"2023-2024": "Player_Name", "Multiplier": "Jerseys Multiplier"})
print(df_sub.head())

# Import into SQLite
df_sub.to_sql('nba_jersey_sales', engine, if_exists='replace', index=False)
print("Imported 'nba_jersey_sales' into SQLite database.\n")

print("All tables have been successfully imported into the SQLite database 'nba_players.db'.")


#### Create an Aggregate Table with ONLY Active Players

Here, we will use SQL to create an aggregate table that will house all information for each player per season. We will populate the full stats table using the primary keys of each individual dataset, and we will use SQL statements to accomplish this. We

Then, we will use an SQL statement to get a subtable of players that excludes inactive/retired players and only has players that are still in the NBA. 

In [ ]:
import sqlite3

# Connect to SQLite database
connection = sqlite3.connect('nba_players.db')  
cursor = connection.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS nba_full_stats (
    Player_ID INTEGER,
    Player TEXT,
    Team TEXT,
    Pos TEXT,
    Age INTEGER,
    G INTEGER,
    GS INTEGER,
    MPG REAL,
    FG REAL,
    FGA REAL,
    "3P" REAL,
    "3PA" REAL,
    "3P%" REAL,
    "2P" REAL,
    "2PA" REAL,
    "2P%" REAL,
    eFG_percent REAL,
    FT REAL,
    FTA REAL,
    "FT%" REAL,
    ORB REAL,
    DRB REAL,
    TRB REAL,
    AST REAL,
    STL REAL,
    BLK REAL,
    TOV REAL,
    PF REAL,
    PTS REAL,
    Total_MP REAL,
    PER REAL,
    TS_percent REAL,
    "3PAr" REAL,
    "FTr" REAL,
    ORB_percent REAL,
    DRB_percent REAL,
    TRB_percent REAL,
    AST_percent REAL,
    STL_percent REAL,
    BLK_percent REAL,
    TOV_percent REAL,
    USG_percent REAL,
    OWS REAL,
    DWS REAL,
    WS REAL,
    WS_per_48 REAL,
    OBPM REAL,
    DBPM REAL,
    BPM REAL,
    VORP REAL,
    p_G TEXT,
    p_GS TEXT,
    p_MPG TEXT,
    p_FG TEXT,
    p_FGA TEXT,
    p_FG_percent TEXT,
    p_3P TEXT,
    p_3PA TEXT,
    p_3P_percent TEXT,
    p_2P TEXT,
    p_2PA TEXT,
    p_2P_percent TEXT,
    p_eFG_percent TEXT,
    p_FT TEXT,
    p_FTA TEXT,
    p_FT_percent TEXT,
    p_ORB TEXT,
    p_DRB TEXT,
    p_TRB TEXT,
    p_AST TEXT,
    p_STL TEXT,
    p_BLK TEXT,
    p_TOV TEXT,
    p_PF TEXT,
    p_PTS TEXT,
    Jerseys_Multiplier REAL,
    Salary REAL,
    Injury_Days INTEGER,
    Valuation_Multiplier REAL
);
"""

cursor.execute(create_table_query)

import sqlite3
connection = sqlite3.connect('nba_players.db')  
cursor = connection.cursor()

insert_query = """
INSERT INTO nba_full_stats (
    Player_ID, Player, Team, Pos, Age, G, GS, Total_MP, PER, TS_percent, 
    "3PAr", "FTr", ORB_percent, DRB_percent, TRB_percent, AST_percent, STL_percent, 
    BLK_percent, TOV_percent, USG_percent, OWS, DWS, WS, WS_per_48, OBPM, DBPM, 
    BPM, VORP
)
SELECT 
    Player_ID, Player, Team, Pos, Age, G, GS, 
    "Total MP", PER, "TS%", "3PAr", "FTr", "ORB%", "DRB%", 
    "TRB%", "AST%", "STL%", "BLK%", "TOV%", "USG%", OWS, 
    DWS, WS, "WS/48", OBPM, DBPM, BPM, VORP
FROM nba_advanced_stats;
"""
cursor.execute(insert_query)
connection.commit()

update_query_game_stat = """
UPDATE nba_full_stats
SET 
    MPG = (SELECT MPG FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    FG = (SELECT FG FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    FGA = (SELECT FGA FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    "3P" = (SELECT "3P" FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    "3PA" = (SELECT "3PA" FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    "3P%" = (SELECT "3P%" FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    "2P" = (SELECT "2P" FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    "2PA" = (SELECT "2PA" FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    "2P%" = (SELECT "2P%" FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    eFG_percent = (SELECT "eFG%" FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    FT = (SELECT FT FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    FTA = (SELECT FTA FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    "FT%" = (SELECT "FT%" FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    ORB = (SELECT ORB FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    DRB = (SELECT DRB FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    TRB = (SELECT TRB FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    AST = (SELECT AST FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    STL = (SELECT STL FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    BLK = (SELECT BLK FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    TOV = (SELECT TOV FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    PF = (SELECT PF FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID),
    PTS = (SELECT PTS FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID)
WHERE EXISTS (SELECT 1 FROM nba_per_game_stats WHERE nba_per_game_stats.Player_ID = nba_full_stats.Player_ID);
"""

cursor.execute(update_query_game_stat)
connection.commit()

update_playoff_query = """
UPDATE nba_full_stats
SET 
    p_G = (SELECT G FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_GS = (SELECT GS FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_MPG = (SELECT MPG FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_FG = (SELECT FG FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_FGA = (SELECT FGA FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_FG_percent = (SELECT "FG%" FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_3P = (SELECT "3P" FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_3PA = (SELECT "3PA" FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_3P_percent = (SELECT "3P%" FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_2P = (SELECT "2P" FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_2PA = (SELECT "2PA" FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_2P_percent = (SELECT "2P%" FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_eFG_percent = (SELECT "eFG%" FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_FT = (SELECT FT FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_FTA = (SELECT FTA FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_FT_percent = (SELECT "FT%" FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_ORB = (SELECT ORB FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_DRB = (SELECT DRB FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_TRB = (SELECT TRB FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_AST = (SELECT AST FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_STL = (SELECT STL FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_BLK = (SELECT BLK FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_TOV = (SELECT TOV FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_PF = (SELECT PF FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID),
    p_PTS = (SELECT PTS FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID)
WHERE EXISTS (SELECT 1 FROM nba_playoff_stats WHERE nba_playoff_stats.Player_ID = nba_full_stats.Player_ID);
"""

cursor.execute(update_playoff_query)
connection.commit()

update_jerseys_query = """
UPDATE nba_full_stats
SET Jerseys_Multiplier = (
    SELECT "Jerseys Multiplier"
    FROM nba_jersey_sales
    WHERE nba_jersey_sales.Player_ID = nba_full_stats.Player_ID
)
WHERE EXISTS (
    SELECT 1
    FROM nba_jersey_sales
    WHERE nba_jersey_sales.Player_ID = nba_full_stats.Player_ID
);
"""

cursor.execute(update_jerseys_query)
connection.commit()

update_salary_query = """
UPDATE nba_full_stats
SET Salary = (
    SELECT Salary
    FROM nba_salaries
    WHERE nba_salaries.Player_ID = nba_full_stats.Player_ID
)
WHERE EXISTS (
    SELECT 1
    FROM nba_salaries
    WHERE nba_salaries.Player_ID = nba_full_stats.Player_ID
);
"""

cursor.execute(update_salary_query)
connection.commit()

update_injury_query = """
UPDATE nba_full_stats
SET Injury_Days = (
    SELECT (Injury_Days)
    FROM nba_injury_days
    WHERE LOWER(TRIM(nba_injury_days.Player)) = LOWER(TRIM(nba_full_stats.Player))
)
WHERE EXISTS (
    SELECT 1
    FROM nba_injury_days
    WHERE LOWER(TRIM(nba_injury_days.Player)) = LOWER(TRIM(nba_full_stats.Player))
);
"""

cursor.execute(update_injury_query)
connection.commit()

update_valuation_query = """
UPDATE nba_full_stats
SET Valuation_Multiplier = (
    SELECT Valuation_Multiplier
    FROM nba_team_valuations
    WHERE LOWER(TRIM(nba_team_valuations.Team)) = LOWER(TRIM(nba_full_stats.Team))
)
WHERE EXISTS (
    SELECT 1
    FROM nba_team_valuations
    WHERE LOWER(TRIM(nba_team_valuations.Team)) = LOWER(TRIM(nba_full_stats.Team))
);
"""

cursor.execute(update_valuation_query)
connection.commit()

# add column for season to be able to explore the data over different time periods
cursor.execute("ALTER TABLE nba_full_stats ADD COLUMN season TEXT")
connection.commit()

update_query = """
UPDATE nba_full_stats
SET season = (
    SELECT season
    FROM nba_advanced_stats
    WHERE nba_full_stats.Player_ID = nba_advanced_stats.Player_ID
)
WHERE Player_ID IN (SELECT Player_ID FROM nba_advanced_stats)
"""

cursor.execute(update_query)
connection.commit()

## Create subtable for only active players
connection = sqlite3.connect('nba_players.db')  
cursor = connection.cursor()

create_subtable_query = """
CREATE TABLE IF NOT EXISTS nba_full_stats_2024 AS
SELECT fs.*
FROM nba_full_stats fs
WHERE LOWER(TRIM(fs.Player)) IN (
    SELECT DISTINCT LOWER(TRIM(Player))
    FROM nba_advanced_stats
    WHERE Season = 2024
)
ORDER BY fs.Player, fs.season;
"""

cursor.execute(create_subtable_query)
connection.commit()

connection.close()